In [3]:
#import package
import matplotlib.pyplot as pp
import pandas as pd
import numpy as np
import math
pp.style.use('ggplot')

In [12]:
HR_KNN=pd.read_csv('/Users/yang/Desktop/2017fall/data mining/project2/IBM-HR-Employee-Attrition.csv')
HR_KNN=HR_KNN.drop(['EmployeeCount','EmployeeNumber',  'Over18', 'StandardHours'], axis=1)


In [13]:
class Classifier_KNN:
    
    #define function to get median number of numeric variable
    def getmedian(self,variable):
        list=variable.tolist()
        if list==[]:
            return []
        sort_list=sorted(list)
        length=len(sort_list)
        if length%2==1:
            return sort_list[int((length+1)/2-1)]
        else:
            return (sort_list[int(length/2-1)]+sort_list[int(length/2)])/2
        
    #define a function to calcualte standard deviation of variable
    def getabsolutestardanddeviation(self,variable):
        sum=0
        median=self.getmedian(variable)
        for item in variable:
            sum+=abs(item-median)
        return sum/len(variable)
    
    # define a function to normalized variable for a dataset
    def normalizecolumn(self,dataset):
        for i in dataset.columns.values:
            vector=dataset[i]
            vector_median=self.getmedian(vector)#get median number 
            vector_asd=self.getabsolutestardanddeviation(vector)#get standard deviation of variable
            dataset[i]=(vector-vector_median)/vector_asd
    #Define Manhattan function. absolute value between two items
    def manhattan(self,vector1,vector2):
        distance=0
        total=0
        n=len(vector1)
        for i in range(n):
            distance+=abs(vector1[i]-vector2[i])#sum of absolute distance
        return distance
    
    # new method
    #K Nearest Neighbors(KNN) 
    #creates a sorted list of items based on their distance to item
    #K=10
    def computenearestneighbor(self,id_number,traindata):
        distances=[]
        for id_other in traindata.index:
            distance=self.manhattan(HR_KNN.loc[id_number],traindata.loc[id_other])
            distances.append((distance,id_other))
        distances.sort()
        neighbor=distances[1:6]
        attrition_vote=0
        nonattrition_vote=0
        for i in pd.DataFrame(neighbor).iloc[:,1]:
            item=HR.loc[i][1]
            if item=='Yes':
                attrition_vote+=1
            elif item=='No':
                nonattrition_vote+=1
            if attrition_vote > nonattrition_vote:
                return('Yes')
            elif attrition_vote < nonattrition_vote:
                return('No')
    
    # test the accuracy of KNN method and print result

    def KNN_accuracy(self,train_data,test_data):
        result={}
        n11=0;n22=0;n12=0;n21=0
        for id in test_data.index:
            classify_result={id:self.computenearestneighbor(id,train_data)}
            result.update(classify_result)
            if classify_result[id]=='Yes'and HR.loc[id][1]=='Yes':
                n11=n11+1
            elif classify_result[id]=='No'and HR.loc[id][1]=='No':
                n22=n22+1
            elif classify_result[id]=='Yes'and HR.loc[id][1]=='No':
                n12=n12+1
            elif classify_result[id]=='No'and HR.loc[id][1]=='Yes':
                n21=n21+1
        n=n11+n12+n21+n22
        p=(n11+n22)/n
        r=((n11+n12)/n)*(n11+n21)+((n21+n22)/n)*(n12+n22)
        pr=r/n
        #print result of classify(Confusion Matrix) and accuracy 
        print( "\n        Classified as: ")
        print("         "+"Yes"+"     No"+"      Total")
        print("------+-------+--------+-------")
        print("|Yes   | ",n11,"   |  ",n21,"   | ",n21+n11,'|')
        print("|No    | ",n12,"  |  ",n22,"  |",n22+n12,'|')
        print("|Total | ",n11+n12,"  |  ",n21+n22,"  |",n,'|')
        print('\n %5.3f percent correct of KNN Method'%p)
        print('percent correct of Random Method:%5.3f'%pr)
        print('Kappa statistic=%5.3f'%((p-pr)/(1-pr)))
    

In [14]:
#Select numeric variables in dataset
object_name=HR_KNN.select_dtypes(['object']).columns.values
HR_KNN=HR_KNN.drop(object_name,axis=1)
HR_KNN.head()

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1102,1,2,2,94,3,2,4,5993,...,3,1,0,8,0,1,6,4,0,5
1,49,279,8,1,3,61,2,2,2,5130,...,4,4,1,10,3,3,10,7,1,7
2,37,1373,2,2,4,92,2,1,3,2090,...,3,2,0,7,3,3,0,0,0,0
3,33,1392,3,4,4,56,3,1,3,2909,...,3,3,0,8,3,3,8,7,3,0
4,27,591,2,1,1,40,3,1,2,3468,...,3,4,1,6,3,3,2,2,2,2


In [15]:
Classify_KNN=Classifier_KNN()
Classify_KNN.normalizecolumn(HR_KNN)
HR_KNN.head()

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.682071,0.856524,-0.952175,-1.317204,-1.1299,1.586616,0.000000,0.000000,1.116173,0.320875,...,0.000000,-2.284382,-1.532847,-0.351296,-3.134328,-4.474886,0.235050,0.346453,-0.505502,0.696517
1,1.773385,-1.493206,0.158696,-2.634409,0.0000,-0.283324,-2.145985,0.000000,-1.116173,0.063040,...,6.504425,1.142191,0.000000,0.000000,0.000000,0.000000,1.175248,1.385812,0.000000,1.393035
2,0.136414,1.630250,-0.793479,-1.317204,1.1299,1.473287,-2.145985,-1.245763,0.000000,-0.845210,...,0.000000,-1.142191,-1.532847,-0.526945,0.000000,0.000000,-1.175248,-1.039359,-0.505502,-1.044776
3,-0.409243,1.684496,-0.634784,1.317204,1.1299,-0.566649,0.000000,-1.245763,0.000000,-0.600520,...,0.000000,0.000000,-1.532847,-0.351296,0.000000,0.000000,0.705149,1.385812,1.011004,-1.044776
4,-1.227728,-0.602422,-0.793479,-2.634409,-2.2598,-1.473287,0.000000,-1.245763,-1.116173,-0.433510,...,0.000000,1.142191,0.000000,-0.702593,0.000000,0.000000,-0.705149,-0.346453,0.505502,-0.348259


In [16]:
train=HR_KNN.sample(frac=0.9)
test=HR_KNN.drop(train.index,axis=0)
Classify_KNN.KNN_accuracy(train,test)

ZeroDivisionError: division by zero

In [6]:
class Classifier_Bayes:
    def __init__(self,train_data):
        self.train=train_data 
        self.attrition=train[train['Attrition']=='Yes'].drop(['Attrition'], axis=1)
        self.nonattrition=train[train['Attrition']=='No'].drop(['Attrition'], axis=1)
        self.attrition_meanandssd={}
        self.nonattrition_meanandssd={}
        self.attrition_pdf={}# pdf of categorical variables in attrition group
        self.nonattrition_pdf={}# pdf of categorical variables in nonattrition group
    def name(self):
        self.attrition.name='attrition'
        self.nonattrition.name='nonattrition'
        
    # define function to calculat stardand deviation of a variable
    def getsd(self,variable):
        list=variable.tolist()
        sum=0
        for item in list:
            sum+=(item-variable.mean())**2 # sum square of difference
        return math.sqrt(sum/(len(list)-1))
    
    # dataset of ssd and mean value for numeric variable in attrition group
    def meanandssd_dictionary(self):
        numeric_variables=self.train.select_dtypes(['int64']).columns.values
        for key in numeric_variables:
            item={key:{'mean':self.attrition[key].mean(),'ssd':self.getsd(self.attrition[key])}}
            self.attrition_meanandssd.update(item)
        for key in numeric_variables:
            item={key:{'mean':self.nonattrition[key].mean(),'ssd':self.getsd(self.nonattrition[key])}}
            self.nonattrition_meanandssd.update(item)
        
    # Dictionary conditional pdf of categorical variables in attrition group
    def categorical_pdf_dictionary(self):
        categorical_variables=self.attrition.select_dtypes(['object']).columns.values
        for variable in categorical_variables:
            result={}
            for key in self.train[variable].unique():
                p=self.attrition[variable].value_counts()[key]/self.attrition.shape[0]
                result.update({key:p}) 
            self.attrition_pdf.update({variable:result})
        for variable in categorical_variables:
            result={}
            for key in self.train[variable].unique():
                p=self.nonattrition[variable].value_counts()[key]/self.nonattrition.shape[0]
                result.update({key:p}) 
            self.nonattrition_pdf.update({variable:result})
 
   
    
                                   
    #define probability density function, input mean, sample standard deviation
    def pdf(self,mean, ssd, x):
        epart=math.e**(-(x-mean)**2/(2*ssd**2))
        return (1/(math.sqrt(2*math.pi)*ssd))*epart
    
    #define function numeric_pdf to calculate probability distribution function of numeric variable
    def numeric_pdf(self,x,variable,condition):
        if self.attrition.name==condition:
            ssd=self.attrition_meanandssd[variable]['ssd']
            mean=self.attrition_meanandssd[variable]['mean']
        else:
            ssd=self.nonattrition_meanandssd[variable]['ssd']
            mean=self.nonattrition_meanandssd[variable]['mean']
        p=self.pdf(x,ssd,mean)
        return p 
    
    def category_pdf(self,x,variable,condition):
        if self.attrition.name==condition:
            p=self.attrition_pdf[variable][x]
        else: p=self.nonattrition_pdf[variable][x]
        return p
    
    # define function to calculate conditional pdf for one observation with numerical 
    # and categorical variable
    def conditionalprobabilty(self,id,condition,testdata):
        a=self.attrition.shape[0]/(self.attrition.shape[0]+self.nonattrition.shape[0])
        if condition==self.attrition.name: p=a# define p(attrition)
        else: p=1-a# define p(nonattrition)
        vector=testdata.drop(['Attrition'], axis=1).loc[id].to_dict()
       #transfer to dictionary
        for key in vector:
            # determine variabel is numerical or categorical
            if self.train[key].dtypes=='int64':
                key_pdf_num=self.numeric_pdf(vector[key],key,condition)
                p=p*key_pdf_num
            else: 
                key_pdf_class=self.category_pdf(vector[key],key,condition)
                p=p*key_pdf_class
        return p
    
    def classify(self,id,testdata):
        p_attri=self.conditionalprobabilty(id,'attrition',testdata)
        p_nonattri=self.conditionalprobabilty(id,'nonattrition',testdata)
        if p_attri>p_nonattri: 
            return {id:'Yes'}
        if p_nonattri>p_attri:
            return {id:'No'}

    def bayes_testaccuracy(self,testdata):
        sum_result={} #create empty dictionary
        n11=0;n22=0;n12=0;n21=0
        for i in testdata.index:
            result=self.classify(i,testdata) #classify item in testing dataset
            sum_result.update(result) #save classify result with id:result in dictionary
            if result[i]=='Yes'and HR.loc[i][1]=='Yes':
                n11=n11+1
            elif result[i]=='No'and HR.loc[i][1]=='No':
                n22=n22+1
            elif result[i]=='Yes'and HR.loc[i][1]=='No':
                n12=n12+1
            elif result[i]=='No'and HR.loc[i][1]=='Yes':
                n21=n21+1
        n=testdata.shape[0]
        p=(n11+n22)/n
        r=((n11+n12)/n)*(n11+n21)+((n21+n22)/n)*(n12+n22)
        pr=r/n
        #print result of classify with accuracy 
        print( "\n        Classified as: ")
        print("         "+"Yes"+"     No"+"      Total")
        print("------+-------+--------+-------")
        print("|Yes   | ",n11,"   |  ",n21,"   |  ",n21+n11,'|')
        print("|No    | ",n12,"  |  ",n22,"  | ",n22+n12,'|')
        print("|Total | ",n11+n12,"  |  ",n21+n22,"  | ",n,'|')
        print('\n %5.3f percent correct'%p)
        print('percent correct:%5.3f based on random method'%pr)
        print('Kappa statistic=%5.3f'%((p-pr)/(1-pr)))

In [7]:
#transfer education level to specific degree. 
HR['Education']=HR['Education'].replace(1,'Below College').replace(2
,'College').replace(3,'Bachelor').replace(4, 'Master').replace(5, 'Doctor')
#transfer EnvironmentSatisfaction   
HR['EnvironmentSatisfaction']=HR['EnvironmentSatisfaction'].replace(1,'Low').replace(2 
,'Medium').replace(3, 'High').replace(4,'Very High')
#transfer JobInvolvement   
HR['JobInvolvement']=HR['JobInvolvement'].replace(1,'Low').replace(2 
,'Medium').replace(3, 'High').replace(4,'Very High')
#transfer JobSatisfaction 
HR['JobSatisfaction']=HR['JobSatisfaction'].replace(1,'Low').replace(2 
,'Medium').replace(3, 'High').replace(4,'Very High')
#transfer performacerating
HR['PerformanceRating']=HR['PerformanceRating'].replace(1,'Low').replace(2 
,'Good').replace(3, 'Excellent').replace(4,'Outstanding')
#tranfer RelationshipSatisfaction 
HR['RelationshipSatisfaction']=HR['RelationshipSatisfaction'].replace(1,'Low').replace(2 
,'Medium').replace(3, 'High').replace(4,'Very High')
#transfer WorkLifeBalance
HR['WorkLifeBalance']=HR['WorkLifeBalance'].replace(1,'Bad').replace(2 
,'Good').replace(3, 'Better').replace(4,'Best')

In [8]:
train=HR.sample(frac=0.9)
test=HR.drop(train.index,axis=0)
attrition=train[train['Attrition']=='Yes'].drop(['Attrition'], axis=1)

In [234]:
train=HR.head(1323)
test=HR.tail(147)

In [147]:
classify_Bayes=Classifier_Bayes

In [18]:
train=HR.sample(frac=0.9)
test=HR.drop(train.index,axis=0)
a=Classifier_Bayes(train)
a.meanandssd_dictionary()
a.name()
a.categorical_pdf_dictionary()
a.bayes_testaccuracy(test)


        Classified as: 
         Yes     No      Total
------+-------+--------+-------
|Yes   |  20    |   12    |   32 |
|No    |  24   |   91   |  115 |
|Total |  44   |   103   |  147 |

 0.755 percent correct
percent correct:0.613 based on random method
Kappa statistic=0.367


In [236]:
a.numeric_pdf(5993,'MonthlyIncome','attrition')

0.00010316298249052564

In [237]:
a.numeric_pdf(5993,'MonthlyIncome','nonattrition')

8.073140185369633e-05

In [238]:
a.category_pdf('Sales','Department','attrition')

0.37614678899082571

In [239]:
a.conditionalprobabilty(1401,'attrition',test)

3.4490052488177171e-49

In [244]:
a.classify(1400,test)

{1400: 'Yes'}

In [255]:
a.bayes_testaccuracy(test)


        Classified as: 
         Yes     No      Total
------+-------+--------+-------
|Yes   |  14    |   5    |   19 |
|No    |  26   |   102   |  128 |
|Total |  40   |   107   |  147 |

 0.789 percent correct
percent correct:0.669 based on random method
Kappa statistic=0.363
